In [0]:
### Ingestion File Production Country

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment =  dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date =  dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

production_country_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("countryId", IntegerType(), True)
])

production_country = spark\
    .read\
    .schema(production_country_schema)\
    .option("multiLine", True)\
    .json(f"{bronze_folder_path}/{v_file_date}/production_country")

production_country_df = production_country\
    .withColumnRenamed("movieId", "movie_id")\
    .withColumnRenamed("countryId", "country_id")\
    .withColumn("environment", lit(v_environment))\
    .withColumn("file_date", lit(v_file_date))

production_country = add_ingestion_date(production_country_df)
# overwrite_partition(production_country, "movie_silver", "production_country", "file_date")

merge_condition = "tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date = src.file_date"
merge_delta_lake(production_country, "movie_silver", "production_country", silver_folder_path, merge_condition, "file_date")


# production_country.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.production_country")


In [0]:
%sql
select file_date, count(1) 
from movie_silver.production_country
group by file_date;

In [0]:
%sql
select *
from movie_silver.production_country;

In [0]:
dbutils.notebook.exit("Success")